# Two beads on a string

Nous allons dans cet exemple procéder à la démonstration la plus complète de la librairie `quantum-string`

On commence par importer les librairies pertinentes

In [19]:
from quantumstring.simulation import Simulation
from quantumstring.particle import Particle, Particles
from quantumstring.edge import ExcitatorSin, MirrorEdge

import numpy as np

import os

On définit un chemin pour la sauvegarde des simulations. On peut prendre le chemin d'accès contenant ce fichier à l'aide de `os.path` ou alors choisir n'importe quel chemin (existant)

In [20]:
mypath = "choose\\your\\path"

On va à présent définir les quantités pertinentes pour la simulation. 

On définit la longueur de la corde $ L $, la tension $ T $ en tout point, et la densité de la corde $ \rho_0 $.

Et on choisit le nombre de cellules $ n_x $ (`space_steps`) que composera la corde.

On définit la célérité $ c $, puis notre distance entre chaque point tel que $ n_x \Delta x = L $
puis on choisit notre $ \Delta t $ qui doit respecter $ c = \frac{\Delta x}{\Delta t} $

In [21]:
duration = 4.0 # duration of simulation [s]
length = 5.0 # [m]
tension = 1.0 # [N]
density = 0.01 # [kg/m]
space_steps = 1024 # string discretisation

celerity = np.sqrt(tension/density)

dx = length/space_steps
dt = dx/celerity
time_steps = int(duration/dt)

On définit les deux instants initiaux de la corde. On la représente par une liste où chaque élément correspond à la valeur du champ en la cellule considérée.

On souhaite avoir une corde au repos. On créé donc une liste contenant uniquement des zéros, de taille `space_steps`

In [22]:
initial_field = [0.0]*space_steps
initial_field_dt = [0.0]*space_steps

A présent nous allons nous occuper des conditions aux bords de la corde. Nous allons utiliser les classes prévues à cet effet de notre module.

Ici, nous allons créer un exitateur sinusoïdal (`ExcitatorSin`) au bout gauche, et un mirroir parfait (`MirrorEdge`) au bord droit. Se référer à la documentation, d'autres conditions existent.

In [23]:
signal_pulsation = 25.0*2.0*np.pi # [rad/s]
edge_left = ExcitatorSin(dt, 0.01, signal_pulsation, 0.0)
edge_right = MirrorEdge()

Pour information, on peut sommer ou multiplier les excitateurs entre eux:

In [ ]:
other_edge = edge_left*ExcitatorSin(dt, 0.01, signal_pulsation/4, 0.0)

Puis nous créons les particules individuellement à l'aide de la classe `Particle`. On donne en argument la position initiale de la particule (id de la cellule), sa vitesse initiale, sa masse $ m $, sa pulsation $ \omega = \sqrt{\frac{K}{m}} $, un booléen (`True`) qui indique si elle est fixée sur la corde, et le nombre total de cellules que compose la corde.

Pour finir, on rassemble toutes ces particules dans un objet `Particles`

In [24]:
pos_a = 0.4
cell_a = int(pos_a*space_steps)
particle_a = Particle(cell_a, 0.0, 0.01, 2*np.pi, True, space_steps, color=(0, 0, 255))

pos_b = 0.6
cell_b = int(pos_b*space_steps)
particle_b = Particle(cell_b, 0.0, 0.02, 2*np.pi, True, space_steps, color=(255, 0, 0))

particles = Particles(particle_a, particle_b)

On va maintenant initialiser la simulation avec la classe `Simulation`, en rentrant les paramètres que nous venons de définir. On oublie pas de `print` la simulation avant de la lancer, pour être sûr que tous nos paramètres nous conviennent

In [26]:
simu = Simulation(dt, time_steps, space_steps, length, density, tension, edge_left, edge_right, initial_field, initial_field_dt, particles)
print(simu)

[SIMULATION]    Δt=0.00048828125s, Δx=0.0048828125m, time steps=8192, string steps (nb discretisation)=1024
[STRING]    L=5.0m, T=1.0N, rho=0.0kg/m, c=10.0m/s ; 2 particles
[PARTICLES]    ;0: m=0.01kg, omega=6.28rad/s;1: m=0.02kg, omega=6.28rad/s;


On peut maintenant lancer la simulation. Le lancement prend en argument obligatoire le chemin pour sauvegarder la simulation.
Des fichiers textes de simulation vont être créés dans le répertoire donné en paramètre. Un fichier pour le champ, un pour la position horizontale des masselottes, un pour l'énergie de chaque cellule de la corde en fonction du temps.

La ligne 0 de ces fichiers correspond à une entête. Ainsi la ligne i du fichier correspond au champ au pas de temps i-1.

Une fois la simulation terminée, la fonction `Simulation.run` renvoie le chemin des fichiers créés (champ/particles/énergie)

In [27]:
field_path, particles_path, energy_path = simu.run(mypath)

[SIMULATION]    Δt=0.00048828125s, Δx=0.0048828125m, time steps=8192, string steps (nb discretisation)=1024
[STRING]    L=5.0m, T=1.0N, rho=0.0kg/m, c=10.0m/s ; 2 particles
[PARTICLES]    ;0: m=0.01kg, omega=6.28rad/s;1: m=0.02kg, omega=6.28rad/s;

video output creation...
video created successfully!


On va maintenant procéder au post processing de la simulation. On instantie une classe `PostProcess`, en donnant en argument les fichiers textes de la simulation ouverts en lecture:

In [ ]:
process = PostProcess(
    open(field_path, "r"),
    open(particles_path, "r"),
    open(energy_path, "r"),
)

Grâce à la classe `PostProcess`, on peut générer:
* Une vidéo de la simulation, fonction `anim`
* Une visualisation de la simulation en 2D colormesh, fonction `plot2d`
* Une visualisation de la simulation en 3D colormesh, fonction `plot3d`
* Les déplacements $ z(t) $ des particules en fonction du temps, avec `plot_particles`
* Des FFT sur des portions de fenêtre, avec `fourier`

In [ ]:
process.anim(mypath, frameskip=5, yscale=5.0, compress=True)

Pour chaque fenêtre, on aura une animation de l'évolution de la FFT au fil du temps dans la fenêtre, et un spectrographe